# Proyecto ISIS-3302 (Modelado, Optimización y Simulación)

- Paulina Arrazola Vernaza - 202020631
- Santiago Alejandro Jaimes Puerto - 201912921
- Nicolás Rincón Sánchez - 202021963

# Etapa III (Metaheurística)

## 1. Implementación de Metaheurísticas (GA)

### 1.1 Importación de Librerías

In [14]:
import pandas as pd
import numpy as np
import random
import math
import requests
import time
import os
import json
import itertools 
from collections import OrderedDict
import sys
import matplotlib.pyplot as plt
import csv

### 1.2 Carga y estructuración de los datos

In [15]:
OSRM_URL = "https://router.project-osrm.org/table/v1/driving/"
C_KM = 20700  # COP/km. Esta constante se saca del análisis realizado en la etapa 1 de costos de transporte 

GA_DATA = {
    "depot_id": None,
    "depot_coords": None,
    "client_ids": [],
    "client_coords": {}, 
    "client_demands": {},
    "all_node_ids": [],
    "all_node_coords": {}, 
    "vehicle_capacity": [], 
    "vehicle_range": [], 
    "num_available_vehicles": 0,
    "distance_matrix": {},
    "cost_matrix": {}, 
}


def load_data(case_name_str):

    # Esctructura
    GA_DATA["depot_id"] = None
    GA_DATA["depot_coords"] = None
    GA_DATA["client_ids"] = []
    GA_DATA["client_coords"] = {}
    GA_DATA["client_demands"] = {}
    GA_DATA["all_node_ids"] = []
    GA_DATA["all_node_coords"] = {}
    GA_DATA["vehicle_capacity"] = [] 
    GA_DATA["vehicle_range"] = []  
    GA_DATA["num_available_vehicles"] = 0
    GA_DATA["distance_matrix"] = {}
    GA_DATA["cost_matrix"] = {}

    base_data_path = "Etapa3/data/Proyecto_A_"
    depots_df_path = f"{base_data_path}CasoBase/depots.csv"

    if case_name_str == "Base":
        clients_df_path = f"{base_data_path}CasoBase/clients.csv"
        vehicles_df_path = f"{base_data_path}CasoBase/vehicles.csv"
    elif case_name_str == "2":
        clients_df_path = f"{base_data_path}Caso2/clients.csv"
        vehicles_df_path = f"{base_data_path}Caso2/vehicles.csv"
    elif case_name_str == "3":
        clients_df_path = f"{base_data_path}Caso3/clients.csv"
        vehicles_df_path = f"{base_data_path}Caso3/vehicles.csv"
    else:
        print(f"Error: Unknown case_name '{case_name_str}'", file=sys.stderr)
        sys.exit(1)

    try:
        depots_df = pd.read_csv(depots_df_path)
        if depots_df.empty:
            raise ValueError(f"Depots CSV ({depots_df_path}) is empty or not found.")
        first_depot_row = depots_df.iloc[0]
        GA_DATA["depot_id"] = f"D{int(first_depot_row['DepotID'])}"
        GA_DATA["depot_coords"] = (float(first_depot_row['Latitude']), float(first_depot_row['Longitude']))
        GA_DATA["all_node_ids"].append(GA_DATA["depot_id"])
        GA_DATA["all_node_coords"][GA_DATA["depot_id"]] = GA_DATA["depot_coords"]

        clients_df = pd.read_csv(clients_df_path)
        if clients_df.empty:
            raise ValueError(f"Clients CSV ({clients_df_path}) is empty or not found.")
        if 'Demand' not in clients_df.columns:
            raise ValueError(f"'Demand' column missing in {clients_df_path}.")
        for idx, row in clients_df.iterrows():
            client_id_val = row['ClientID']
            if isinstance(client_id_val, float) and client_id_val.is_integer():
                client_id_val = int(client_id_val)
            client_id_str = f"C{client_id_val}"
            GA_DATA["client_ids"].append(client_id_str)
            GA_DATA["all_node_ids"].append(client_id_str)
            client_coords = (float(row['Latitude']), float(row['Longitude']))
            GA_DATA["client_coords"][client_id_str] = client_coords
            GA_DATA["all_node_coords"][client_id_str] = client_coords
            GA_DATA["client_demands"][client_id_str] = float(row['Demand'])
        print(f"Loaded {len(GA_DATA['client_ids'])} clients for Case {case_name_str}. First few: {GA_DATA['client_ids'][:5]}")

        # Sección de carga de vehículos
        vehicles_df = pd.read_csv(vehicles_df_path)
        if vehicles_df.empty:
            raise ValueError(f"Vehicles CSV ({vehicles_df_path}) is empty or not found.")
        if 'Range' not in vehicles_df.columns or 'Capacity' not in vehicles_df.columns:
            raise ValueError(f"'Range' or 'Capacity' column missing in {vehicles_df_path}.")
        
        # Cargar rangos y capacidades como listas
        GA_DATA["vehicle_range"] = vehicles_df['Range'].tolist()
        GA_DATA["vehicle_capacity"] = vehicles_df['Capacity'].tolist()
        GA_DATA["num_available_vehicles"] = len(vehicles_df)

    except FileNotFoundError as e:
        print(f"File not found. {e}", file=sys.stderr)
        sys.exit(1)
    except ValueError as e: 
        print(f"Name issue. {e}", file=sys.stderr)
        sys.exit(1)
    except KeyError as e: 
        print(f"Missing expected column in CSV. Problematic column: {e}", file=sys.stderr)
        sys.exit(1)
    except Exception as e:
        print(f"An unexpected error occurred during data loading: {e}", file=sys.stderr)
        sys.exit(1)


    # Calculo OSRM de distancias
    print("--- Calculating Distance Matrix using OSRM ---")
    N_ga = GA_DATA["all_node_ids"]
    coords_ga = GA_DATA["all_node_coords"]
    if not GA_DATA["client_ids"]:
         print(f"No clients were loaded for Case {case_name_str}. GA cannot proceed.", file=sys.stderr)
    CACHE_FILE_GA = f'Etapa3/cache/osrm_cache_ga_{case_name_str.lower().replace(" ", "_")}.json'
    os.makedirs(os.path.dirname(CACHE_FILE_GA), exist_ok=True)

    dist_cache = {}
    if os.path.exists(CACHE_FILE_GA):
        try:
            with open(CACHE_FILE_GA, 'r') as f:
                raw_cache = json.load(f)
            dist_cache = {tuple(k.split("|")): v for k, v in raw_cache.items()}
            print(f"  Distances loaded from GA cache ({len(dist_cache)} pairs): {CACHE_FILE_GA}")
        except json.JSONDecodeError:
            dist_cache = {}
    
    all_pairs_to_calculate = list(itertools.product(N_ga, N_ga))
    missing_pairs = [p for p in all_pairs_to_calculate if p not in dist_cache and p[0] != p[1]] 

    MAX_COORDS_OSRM = 100 

    # Revisa si hay pares de nodos que faltan en el cache
    if missing_pairs:
        BATCH_SIZE = max(1, MAX_COORDS_OSRM // 2) 
        
        for i_start_node_batch_start in range(0, len(N_ga), BATCH_SIZE):
            sources_nodes = N_ga[i_start_node_batch_start : i_start_node_batch_start + BATCH_SIZE]
            
            for j_start_node_batch_start in range(0, len(N_ga), BATCH_SIZE):
                dests_nodes = N_ga[j_start_node_batch_start : j_start_node_batch_start + BATCH_SIZE]
                
                current_query_nodes = list(OrderedDict.fromkeys(sources_nodes + dests_nodes)) 
                if not current_query_nodes: continue

                node_to_idx_map = {node_id: k for k, node_id in enumerate(current_query_nodes)}
                query_coords_str = ";".join([f"{coords_ga[n][1]},{coords_ga[n][0]}" for n in current_query_nodes])
                
                src_indices_in_query = ";".join(str(node_to_idx_map[n]) for n in sources_nodes if n in node_to_idx_map)
                dst_indices_in_query = ";".join(str(node_to_idx_map[n]) for n in dests_nodes if n in node_to_idx_map)

                if not src_indices_in_query or not dst_indices_in_query: continue

                url_batch = f"{OSRM_URL}{query_coords_str}"
                params_batch = {
                    "sources": src_indices_in_query,
                    "destinations": dst_indices_in_query,
                    "annotations": "distance"
                }
                # Hace la peticiín a OSRM
                try:
                    r = requests.get(url_batch, params=params_batch, timeout=180)
                    r.raise_for_status()
                    matrix_data = r.json()
                    if "distances" not in matrix_data or not matrix_data["distances"]:
                        for u_node_s in sources_nodes:
                            for v_node_d in dests_nodes:
                                if (u_node_s, v_node_d) not in dist_cache: dist_cache[(u_node_s, v_node_d)] = float('inf')
                        continue
                    
                    batch_distances = matrix_data["distances"]
                    for src_idx_local, u_node_s in enumerate(sources_nodes):
                        for dst_idx_local, v_node_d in enumerate(dests_nodes):
                            if u_node_s == v_node_d:
                                dist_cache[(u_node_s, v_node_d)] = 0.0
                                continue
                            if src_idx_local < len(batch_distances) and dst_idx_local < len(batch_distances[src_idx_local]):
                                dist_val = batch_distances[src_idx_local][dst_idx_local]
                                dist_cache[(u_node_s, v_node_d)] = float('inf') if dist_val is None else dist_val / 1000.0
                            else:
                                dist_cache[(u_node_s, v_node_d)] = float('inf')

                except requests.exceptions.Timeout:
                    print(f"    Error: OSRM request timed out for batch. Assigning Inf.")
                    for u_node_s in sources_nodes:
                        for v_node_d in dests_nodes:
                            if (u_node_s, v_node_d) not in dist_cache: dist_cache[(u_node_s, v_node_d)] = float('inf')
                except requests.exceptions.RequestException as e_req:
                    print(f"    Error: OSRM request failed for batch: {e_req}. Assigning Inf.")
                    for u_node_s in sources_nodes:
                        for v_node_d in dests_nodes:
                            if (u_node_s, v_node_d) not in dist_cache: dist_cache[(u_node_s, v_node_d)] = float('inf')
        
        # Guarda el cache actualizado
        ordered_dist_cache_to_save = OrderedDict()
        sorted_nodes_for_cache = sorted(N_ga) 
        for u_node in sorted_nodes_for_cache:
            for v_node in sorted_nodes_for_cache:
                ordered_dist_cache_to_save[f"{u_node}|{v_node}"] = dist_cache.get((u_node, v_node), float('inf') if u_node !=v_node else 0.0)

        with open(CACHE_FILE_GA, 'w') as f:
            json.dump(ordered_dist_cache_to_save, f, indent=2)
        print(f"  GA OSRM cache updated: {CACHE_FILE_GA}")
    else:
        print(f"  No missing distance pairs for Case {case_name_str}")

    # Poblar matrices de distancia y costo
    for u_node in N_ga:
        for v_node in N_ga:
            if u_node == v_node:
                GA_DATA["distance_matrix"][(u_node, v_node)] = 0.0
                GA_DATA["cost_matrix"][(u_node, v_node)] = 0.0
            else:
                dist = dist_cache.get((u_node, v_node), float('inf'))
                GA_DATA["distance_matrix"][(u_node, v_node)] = dist
                GA_DATA["cost_matrix"][(u_node, v_node)] = dist * C_KM
    
    print(f"Distance and cost matrices populated for case {case_name_str}. {len(GA_DATA['distance_matrix'])} entries.")
    if GA_DATA["depot_id"] in GA_DATA["client_ids"]: 
        print(f"Depot ID {GA_DATA['depot_id']} is also in client_ids list for Case {case_name_str}!", file=sys.stderr)
        sys.exit(1)
    print(f"--- DATA LOADING COMPLETE FOR CASE {case_name_str} ---")




In [16]:
# Visualización de los datos cargados en GA_DATA
def visualize_ga_data():
    print("--- Visualizing GA_DATA ---")
    df = pd.DataFrame.from_dict(GA_DATA, orient='index')
    print(df)

visualize_ga_data()

--- Visualizing GA_DATA ---
                           0
depot_id                None
depot_coords            None
client_ids                []
client_coords             {}
client_demands            {}
all_node_ids              []
all_node_coords           {}
vehicle_capacity          []
vehicle_range             []
num_available_vehicles     0
distance_matrix           {}
cost_matrix               {}


### 1.3 Algorítmo Genético (cromosomas, fitness, cruce, mutación, etc.)

In [17]:
# Decodifica un cromosoma en una lista de rutas. 
# Cada ruta comienza y termina en el depósito y respeta la capacidad y rango específicos del vehículo
def decode_chromosome_to_routes(chromosome, data):
    routes = []
    current_route_nodes = [data["depot_id"]]
    current_route_capacity_load = 0
    current_route_distance_travelled = 0
    current_vehicle_idx = 0
    
    total_solution_cost = 0
    feasibility_penalty = 0
    clients_to_visit_ordered = list(chromosome)
    chromosome_client_idx = 0

    # Ordenar los clientes para visitar en el orden del cromosoma
    while chromosome_client_idx < len(clients_to_visit_ordered):
        if current_vehicle_idx >= data["num_available_vehicles"]:
            # Si no hay más vehículos disponibles, penalizar por falta de viabilidad 
            remaining_unserved = len(clients_to_visit_ordered) - chromosome_client_idx
            feasibility_penalty += remaining_unserved * 10_000_000
            break

        
        client_id_to_try = clients_to_visit_ordered[chromosome_client_idx]
        client_demand = data["client_demands"].get(client_id_to_try, float('inf'))
        current_vehicle_capacity = data["vehicle_capacity"][current_vehicle_idx]
        current_vehicle_range = data["vehicle_range"][current_vehicle_idx]
        needs_new_vehicle = (len(current_route_nodes) == 1 and current_route_nodes[0] == data["depot_id"])
        
        # Si es el primer cliente de la ruta, verificar si se necesita un nuevo vehículo
        if needs_new_vehicle:
            current_route_capacity_load = 0
            current_route_distance_travelled = 0

        can_add_this_client = True
        last_node = current_route_nodes[-1]
        dist_to_client = data["distance_matrix"].get((last_node, client_id_to_try), float('inf'))
        dist_to_depot = data["distance_matrix"].get((client_id_to_try, data["depot_id"]), float('inf'))

        # Verificar capacidad y distancia
        if dist_to_client == float('inf') or dist_to_depot == float('inf'):
            can_add_this_client = False
            feasibility_penalty += 20_000_000

        # Calcular nueva carga y distancia si se agrega el cliente en la ruta
        new_load = current_route_capacity_load + client_demand
        new_distance = current_route_distance_travelled + dist_to_client + dist_to_depot

        if new_load > current_vehicle_capacity:
            can_add_this_client = False
        
        if new_distance > current_vehicle_range:
            can_add_this_client = False

        # Si las verificaciones son correctas, se agrega el cliente a la ruta
        if can_add_this_client:
            current_route_nodes.append(client_id_to_try)
            current_route_capacity_load = new_load
            current_route_distance_travelled += dist_to_client
            chromosome_client_idx += 1
        else:
            if needs_new_vehicle:
                # Penalizar por no poder servir al cliente con el vehículo actual
                feasibility_penalty += 50_000_000
                # Si no se puede servir al cliente con el vehículo actual se pasa al siguiente vehículo
                chromosome_client_idx += 1
            else:
                if len(current_route_nodes) > 1:
                    dist_return = data["distance_matrix"].get((current_route_nodes[-1], data["depot_id"]), float('inf'))
                    if dist_return == float('inf'):
                        # Penalizar por no regresar al depósito
                        feasibility_penalty += 20_000_000
                    else:
                        final_route_dist = current_route_distance_travelled + dist_return
                        # Costo de la ruta final que se calcula con la distancia final y el costo fijo por km
                        total_solution_cost += final_route_dist * C_KM
                    routes.append((current_vehicle_idx, list(current_route_nodes) + [data["depot_id"]]))
                current_route_nodes = [data["depot_id"]]
                current_route_capacity_load = 0
                current_route_distance_travelled = 0
                current_vehicle_idx += 1

    # Completar última ruta si es necesario
    if len(current_route_nodes) > 1:
        dist_return_last = data["distance_matrix"].get((current_route_nodes[-1], data["depot_id"]), float('inf'))
        if dist_return_last == float('inf'):
            # Penalizar por no ser capaz de regresar al depósito
            feasibility_penalty += 20_000_000
        else:
            final_route_dist = current_route_distance_travelled + dist_return_last
            total_solution_cost += final_route_dist * C_KM
        routes.append((current_vehicle_idx, list(current_route_nodes) + [data["depot_id"]]))

    # Verificar clientes no servidos
    served_clients = set()
    for _, route in routes:
        for node in route:
            if node != data["depot_id"]:
                served_clients.add(node)

    # Penalizar por clientes no servidos
    for client in chromosome:
        if client not in served_clients:
            feasibility_penalty += 1_000_000

    return routes, total_solution_cost, feasibility_penalty

# Calcula el fitness de un indiciduo
# Menos fitness (costo) es mejor
def calculate_cvrp_fitness(chromosome_individual, data):
    # Asegurar que el cromosoma contiende todos los clientes exactamente una vez
    if sorted(chromosome_individual) != sorted(data["client_ids"]):
        return float('inf') # Cromosoma invalido se penaliza con infinito

    routes, decoded_cost, penalty = decode_chromosome_to_routes(chromosome_individual, data)
    fitness = decoded_cost + penalty
    return fitness

# Genera una población inicial de cromosomas aleatorios
def generate_initial_population(size, client_ids_list):
    population = []
    for _ in range(size):
        # Copia de la lista de clientes que se mezcla
        individual = list(client_ids_list) 
        random.shuffle(individual)
        population.append(individual)
    return population

# Selección por torneo
def selection_by_tournament(population, fitness_func, data, k=3):
    selected_parents = []
    for _ in range(len(population)):
        participants = random.sample(population, k)
        # Evaluar la aptitud de los participantes
        participant_fitnesses = [fitness_func(ind, data) for ind in participants]
        winner = participants[participant_fitnesses.index(min(participant_fitnesses))]
        selected_parents.append(list(winner))
    return selected_parents

# Se utiliza Order Crossover (OX) para combinar dos padres en un hijo
def order_crossover(parent1, parent2):
    size = len(parent1)
    child = [None] * size
    
    # Se seleccionan puntos de inicio y fin aleatorios
    start, end = sorted(random.sample(range(size), 2))


    child[start:end+1] = parent1[start:end+1]
    parent2_elements = [item for item in parent2 if item not in child[start:end+1]]
    current_pos = 0
    for i in range(size):
        if child[i] is None:
            child[i] = parent2_elements[current_pos]
            current_pos += 1
    return child

# Realiza una mutación de intercambio en un individuo con una probabilidad dada
def swap_mutation(individual, mutation_rate):
    mutated_individual = list(individual)
    if random.random() < mutation_rate:
        idx1, idx2 = random.sample(range(len(mutated_individual)), 2)
        mutated_individual[idx1], mutated_individual[idx2] = mutated_individual[idx2], mutated_individual[idx1]
    return mutated_individual

# Algortimo genético (GA) para resolver el CVRO
def run_genetic_algorithm_cvrp(data, pop_size, generations, mutation_rate, tournament_k):
    client_ids_list = data["client_ids"]

    # Inicaliza la población, el mejor fitness y el mejor cromosoma
    population = generate_initial_population(pop_size, client_ids_list)
    best_overall_fitness = float('inf')
    best_overall_chromosome = None
    
    generation_log = []

    for gen in range(generations):
        # Evalua el fitness para la población actual
        fitness_values = [calculate_cvrp_fitness(ind, data) for ind in population]
        
        # Encuenta el mejor cromosoma de la generación actual
        min_fitness_current_gen = min(fitness_values)
        best_chromosome_current_gen = population[fitness_values.index(min_fitness_current_gen)]
        
        if min_fitness_current_gen < best_overall_fitness:
            best_overall_fitness = min_fitness_current_gen
            best_overall_chromosome = list(best_chromosome_current_gen) 
        
        avg_fitness_current_gen = sum(fitness_values) / len(fitness_values)
        generation_log.append((gen + 1, best_overall_fitness, avg_fitness_current_gen))
        print(f"Generation {gen+1}/{generations} | Best Fitness: {best_overall_fitness:.2f} | Avg Fitness: {avg_fitness_current_gen:.2f}")
        
        # Selección
        parents = selection_by_tournament(population, calculate_cvrp_fitness, data, k=tournament_k)
        
        # Cruce y mutación
        next_population = []
        
        for i in range(0, pop_size, 2):
            if i + 1 < len(parents):
                p1 = parents[i]
                p2 = parents[i+1] # Si el tamaño de la población es impar, el último padre no se cruza
                
                child1 = order_crossover(p1, p2)
                child2 = order_crossover(p2, p1)
                
                child1_mutated = swap_mutation(child1, mutation_rate)
                child2_mutated = swap_mutation(child2, mutation_rate)
                
                next_population.append(child1_mutated)
                next_population.append(child2_mutated)
            elif i < len(parents): 
                next_population.append(parents[i])

        # Si la población resultante es menor que el tamaño deseado, se añaden mutaciones de padres aleatorios
        if len(next_population) < pop_size : 
             if parents:
                 needed = pop_size - len(next_population)
                 for _ in range(needed):
                    next_population.append(swap_mutation(random.choice(parents), mutation_rate))


        population = next_population[:pop_size] # Asegurar que la población

    print("--- Genetic Algorithm Finished ---")
    return best_overall_chromosome, best_overall_fitness, generation_log

## 2. Calibración y experimentación

### 2.1 Plan experimental para Calibracion de Parametros

Para el proceso de calibración de parámetros del algoritmo genético aplicado al CVRP, se ha diseñado un plan experimental estructurado basado en un enfoque factorial completo. Este método permite explorar sistemáticamente el espacio de parámetros clave (tamaño de población, tasa de mutación, número de generaciones y tamaño del torneo de selección) evaluando todas sus posibles combinaciones dentro de rangos predefinidos. Para garantizar la robustez estadística de los resultados, cada configuración se ejecutará en tres corridas independientes utilizando diferentes semillas aleatorias (42, 123 y 7890), lo que mitiga el efecto de la estocasticidad inherente a los algoritmos metaheurísticos. Durante la experimentación se registrarán métricas cruciales como el valor de la función objetivo (costo total de las rutas), tiempo de CPU, y estadísticas descriptivas (mejor valor, peor valor, media y desviación estándar) para cada configuración. Este rigor metodológico permitirá identificar la combinación óptima de parámetros que equilibre la calidad de las soluciones con la eficiencia computacional, mejorando así el rendimiento general del algoritmo para los diversos casos de prueba considerados en este estudio.

### 2.2 Experimentación con diferentes semillas
### 2.3 Mejor configuración y estadísticas relevantes

In [ ]:
# CODIGO DE EXPERIMENTACION EL 2.2 Y 2.3 VAN ACA

# 4. CALIBRACIÓN Y EXPERIMENTACIÓN

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from tqdm import tqdm
import random

def save_solution_in_requested_format(routes, depot_id, filename="best_solution_formatted.csv"):
    """
    Guarda la solución de rutas en el formato específico solicitado:
    VehicleId, DepotId, InitialLoad, RouteSequence, ClientsServed, DemandsSatisfied,
    TotalDistance, TotalTime, FuelCost
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Escribir encabezado
        writer.writerow(['VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence', 'ClientsServed', 
                        'DemandsSatisfied', 'TotalDistance', 'TotalTime', 'FuelCost'])
        
        # Para cada ruta
        for i, (vehicle_idx, route) in enumerate(routes):
            if not route or len(route) <= 2:
                continue
                
            # Identificadores
            vehicle_id = f"VEH{(i+1):03d}"
            depot_id_str = depot_id
            
            # Calcular carga inicial (suma de las demandas de todos los clientes en la ruta)
            clients_in_route = [node for node in route if node != depot_id]
            initial_load = sum(GA_DATA["client_demands"].get(client, 0) for client in clients_in_route)
            
            # Secuencia de la ruta
            route_sequence = " - ".join(route)
            
            # Clientes atendidos (número)
            clients_served = len(clients_in_route)
            
            # Demandas satisfechas
            demands_satisfied = " - ".join([str(int(GA_DATA["client_demands"].get(client, 0))) 
                                          for client in clients_in_route])
            
            # Cálculo de distancia total
            total_distance = 0
            for k_idx in range(len(route) - 1):
                u, v = route[k_idx], route[k_idx+1]
                total_distance += GA_DATA["distance_matrix"].get((u, v), 0)
            
            # Tiempo total (estimando un tiempo promedio de 2.5 minutos por km)
            # Este es un valor arbitrario, ajusta según tus necesidades
            avg_time_per_km = 2.5  # minutos por km
            total_time = total_distance * avg_time_per_km
            
            # Costo de combustible (usando la constante C_KM)
            fuel_cost = int(total_distance * C_KM)
            
            # Escribir la fila
            writer.writerow([
                vehicle_id,
                depot_id_str,
                int(initial_load),
                route_sequence,
                clients_served,
                demands_satisfied,
                f"{total_distance:.1f}",
                f"{total_time:.1f}",
                fuel_cost
            ])
    
    print(f"Solución guardada en formato solicitado: {filename}")


def calibrate_genetic_algorithm(data_case="Base"):
    """
    Ejecuta un proceso estructurado de calibración para el algoritmo genético
    usando un diseño factorial completo y múltiples corridas por configuración.
    """
    # Cargar datos para el caso seleccionado
    print(f"\n--- INICIANDO CALIBRACIÓN PARA CASO: {data_case} ---\n")
    load_data(data_case)
    
    # Definir parámetros a calibrar y sus rangos
    parameter_ranges = {
        'population_size': [50, 100, 150],
        'generations': [100, 200],
        'mutation_rate': [0.05, 0.10, 0.15],
        'tournament_k': [3, 5, 7]
    }
    
    # Crear todas las combinaciones posibles de parámetros
    param_keys = list(parameter_ranges.keys())
    param_values = list(parameter_ranges.values())
    configurations = list(product(*param_values))
    
    # Preparar DataFrame para almacenar resultados
    results = []
    
    # Semillas para reproducibilidad
    seeds = [42, 123, 7890]  # Diferentes semillas para cada ejecución
    num_runs = len(seeds)  # 3 ejecuciones independientes por configuración
    
    print(f"Plan experimental: {len(configurations)} configuraciones × {num_runs} corridas = {len(configurations)*num_runs} experimentos")
    print("Parámetros evaluados:", parameter_ranges)
    
    # Ejecutar experimentos para cada configuración
    for config_id, config in enumerate(tqdm(configurations, desc="Progreso de calibración")):
        config_params = dict(zip(param_keys, config))
        print(f"\nConfiguración #{config_id+1}/{len(configurations)}: {config_params}")
        
        for run, seed in enumerate(seeds):
            print(f"  Corrida {run+1}/{num_runs} (semilla: {seed})")
            
            # Establecer semillas para reproducibilidad
            random.seed(seed)
            np.random.seed(seed)
            
            # Medir tiempo de CPU
            start_time = time.time()
            
            # Ejecutar algoritmo con los parámetros actuales
            best_chromosome, best_fitness, ga_log = run_genetic_algorithm_cvrp(
                GA_DATA,
                pop_size=config_params['population_size'],
                generations=config_params['generations'],
                mutation_rate=config_params['mutation_rate'],
                tournament_k=config_params['tournament_k']
            )
            
            # Decodificar solución para obtener rutas y calcular métricas
            final_routes, final_cost, final_penalty = decode_chromosome_to_routes(best_chromosome, GA_DATA)
            
            # Calcular tiempo transcurrido
            end_time = time.time()
            cpu_time = end_time - start_time
            
            # Calcular métricas adicionales
            num_vehicles = len(final_routes)
            total_distance = sum([
                sum([GA_DATA["distance_matrix"].get((route[i], route[i+1]), 0) 
                for i in range(len(route)-1)])
                for _, route in final_routes
            ])
            
            # Almacenar resultados
            results.append({
                'config_id': config_id + 1,
                'run': run + 1,
                'seed': seed,
                'cpu_time': cpu_time,
                'objective_value': best_fitness,
                'final_cost': final_cost,
                'penalty': final_penalty,
                'num_vehicles': num_vehicles,
                'total_distance': total_distance,
                'converged_at': len(ga_log),
                **config_params  # Incluir todos los parámetros
            })
    
    # Convertir a DataFrame
    results_df = pd.DataFrame(results)
    
    # Calcular estadísticas por configuración
    stats_df = results_df.groupby('config_id').agg(
        mean_obj=('objective_value', 'mean'),
        std_obj=('objective_value', 'std'),
        min_obj=('objective_value', 'min'),  # mejor valor (minimización)
        max_obj=('objective_value', 'max'),  # peor valor (minimización)
        mean_time=('cpu_time', 'mean'),
        mean_vehicles=('num_vehicles', 'mean'),
        mean_distance=('total_distance', 'mean')
    ).reset_index()
    
    # Unir estadísticas con información de parámetros
    for param in param_keys:
        stats_df[param] = [results_df[results_df['config_id'] == cid][param].iloc[0] for cid in stats_df['config_id']]
    
    # Encontrar la mejor configuración (problema de minimización)
    best_config_idx = stats_df['mean_obj'].idxmin()
    best_config = stats_df.iloc[best_config_idx]
    
    # Mostrar resultados
    print("\n" + "="*60)
    print("RESULTADOS DE LA CALIBRACIÓN")
    print("="*60)
    print("\nMejor configuración encontrada:")
    print(f"  Config #{best_config['config_id']}")
    for param in param_keys:
        print(f"  {param}: {best_config[param]}")
    print("\nEstadísticas de la mejor configuración:")
    print(f"  Valor objetivo promedio: {best_config['mean_obj']:,.2f}")
    print(f"  Desviación estándar: {best_config['std_obj']:,.2f}")
    print(f"  Mejor valor: {best_config['min_obj']:,.2f}")
    print(f"  Peor valor: {best_config['max_obj']:,.2f}")
    print(f"  Tiempo CPU promedio: {best_config['mean_time']:.2f} segundos")
    print(f"  Vehículos promedio: {best_config['mean_vehicles']:.2f}")
    print(f"  Distancia total promedio: {best_config['mean_distance']:.2f} km")
    
    # Guardar resultados en CSV
    results_df.to_csv(f'calibracion_resultados_{data_case}.csv', index=False)
    stats_df.to_csv(f'calibracion_estadisticas_{data_case}.csv', index=False)
    print(f"\nResultados guardados en 'calibracion_resultados_{data_case}.csv' y 'calibracion_estadisticas_{data_case}.csv'")
    
    # Visualizaciones
    plt.figure(figsize=(15, 12))
    
    # Gráfico 1: Valor objetivo promedio por configuración
    plt.subplot(2, 2, 1)
    ax = sns.barplot(x='config_id', y='mean_obj', data=stats_df)
    plt.title('Valor Objetivo Promedio por Configuración', fontsize=12)
    plt.xlabel('ID de Configuración')
    plt.ylabel('Valor Objetivo (COP)')
    plt.axhline(y=best_config['mean_obj'], color='red', linestyle='--')
    plt.xticks(rotation=90)
    
    # Destacar la mejor configuración
    best_bar = ax.patches[best_config_idx]
    best_bar.set_facecolor('green')
    
    # Gráfico 2: Tiempo CPU promedio por configuración
    plt.subplot(2, 2, 2)
    sns.barplot(x='config_id', y='mean_time', data=stats_df)
    plt.title('Tiempo CPU Promedio por Configuración', fontsize=12)
    plt.xlabel('ID de Configuración')
    plt.ylabel('Tiempo (segundos)')
    plt.xticks(rotation=90)
    
    # Gráfico 3: Distribución de valores objetivo por configuración
    plt.subplot(2, 2, 3)
    sns.boxplot(x='config_id', y='objective_value', data=results_df)
    plt.title('Distribución de Valores Objetivo', fontsize=12)
    plt.xlabel('ID de Configuración')
    plt.ylabel('Valor Objetivo (COP)')
    plt.xticks(rotation=90)
    
    # Gráfico 4: Análisis de parámetros para el valor objetivo
    plt.subplot(2, 2, 4)
    
    # Crear datos para el gráfico de efectos principales
    param_data = []
    for param in param_keys:
        for val in parameter_ranges[param]:
            subset = results_df[results_df[param] == val]
            param_data.append({
                'parameter': param,
                'value': str(val),  # Convertir a string para que funcione con cualquier tipo
                'objective_mean': subset['objective_value'].mean()
            })
    param_df = pd.DataFrame(param_data)
    
    # Graficar efectos principales
    sns.lineplot(x='value', y='objective_mean', hue='parameter', markers=True, 
                style='parameter', dashes=False, data=param_df)
    plt.title('Efecto de los Parámetros en el Valor Objetivo', fontsize=12)
    plt.ylabel('Valor Objetivo Promedio')
    plt.xlabel('Valor del Parámetro')
    plt.legend(title='Parámetro')
    
    plt.tight_layout()
    plt.savefig(f'calibracion_graficos_{data_case}.png', dpi=300)
    plt.show()
    
    print(f"Visualizaciones guardadas en 'calibracion_graficos_{data_case}.png'")
    
    # Devolver la mejor configuración de parámetros
    best_params = {param: best_config[param] for param in param_keys}
    return best_params, results_df, stats_df, final_routes

# Ejemplo de uso:
if __name__ == "__main__":
    
    for case_name in ["Base", "2", "3"]:
        print(f"\n--- CALIBRATING FOR CASE: {case_name} ---")
        
        
        best_params, results_df, stats_df, final_routes = calibrate_genetic_algorithm(case_name)
        print(f"Best parameters for case {case_name}: {best_params}")
        print(f"Results saved for case {case_name}.")

        save_solution_in_requested_format(final_routes, GA_DATA["depot_id"],
                         filename=f"best_solution_case_{case_name}_formatted.csv")



--- CALIBRATING FOR CASE: Base ---

--- INICIANDO CALIBRACIÓN PARA CASO: Base ---

Loaded 24 clients for Case Base. First few: ['C1', 'C2', 'C3', 'C4', 'C5']
--- Calculating Distance Matrix using OSRM ---
  Distances loaded from GA cache (625 pairs): Etapa3/cache/osrm_cache_ga_base.json
  No missing distance pairs for Case Base
Distance and cost matrices populated for case Base. 625 entries.
--- DATA LOADING COMPLETE FOR CASE Base ---
Plan experimental: 54 configuraciones × 3 corridas = 162 experimentos
Parámetros evaluados: {'population_size': [50, 100, 150], 'generations': [100, 200], 'mutation_rate': [0.05, 0.1, 0.15], 'tournament_k': [3, 5, 7]}


Progreso de calibración:   0%|          | 0/54 [00:00<?, ?it/s]


Configuración #1/54: {'population_size': 50, 'generations': 100, 'mutation_rate': 0.05, 'tournament_k': 3}
  Corrida 1/3 (semilla: 42)
Generation 1/100 | Best Fitness: 8123394.15 | Avg Fitness: 9711398.79
Generation 2/100 | Best Fitness: 8123394.15 | Avg Fitness: 9548980.67
Generation 3/100 | Best Fitness: 7511630.49 | Avg Fitness: 9149562.45
Generation 4/100 | Best Fitness: 7511630.49 | Avg Fitness: 8888428.02
Generation 5/100 | Best Fitness: 7347429.81 | Avg Fitness: 8734893.39
Generation 6/100 | Best Fitness: 7347429.81 | Avg Fitness: 8653099.45
Generation 7/100 | Best Fitness: 6969961.17 | Avg Fitness: 8515468.88
Generation 8/100 | Best Fitness: 6969961.17 | Avg Fitness: 8391033.31
Generation 9/100 | Best Fitness: 6969961.17 | Avg Fitness: 8095801.75
Generation 10/100 | Best Fitness: 6961563.18 | Avg Fitness: 8117238.51
Generation 11/100 | Best Fitness: 6739574.31 | Avg Fitness: 8069207.51
Generation 12/100 | Best Fitness: 6739574.31 | Avg Fitness: 7797965.68
Generation 13/100 | B

Progreso de calibración:   2%|▏         | 1/54 [00:08<07:37,  8.63s/it]

Generation 96/100 | Best Fitness: 5645202.57 | Avg Fitness: 5700576.97
Generation 97/100 | Best Fitness: 5645202.57 | Avg Fitness: 5667270.84
Generation 98/100 | Best Fitness: 5645202.57 | Avg Fitness: 5722477.20
Generation 99/100 | Best Fitness: 5645202.57 | Avg Fitness: 5752911.21
Generation 100/100 | Best Fitness: 5645202.57 | Avg Fitness: 5691156.40
--- Genetic Algorithm Finished ---

Configuración #2/54: {'population_size': 50, 'generations': 100, 'mutation_rate': 0.05, 'tournament_k': 5}
  Corrida 1/3 (semilla: 42)
Generation 1/100 | Best Fitness: 8123394.15 | Avg Fitness: 9711398.79
Generation 2/100 | Best Fitness: 8123394.15 | Avg Fitness: 9576863.20
Generation 3/100 | Best Fitness: 7418393.55 | Avg Fitness: 9147802.33
Generation 4/100 | Best Fitness: 7418393.55 | Avg Fitness: 8841141.89
Generation 5/100 | Best Fitness: 7378935.21 | Avg Fitness: 8559353.25
Generation 6/100 | Best Fitness: 7356999.42 | Avg Fitness: 8517977.55
Generation 7/100 | Best Fitness: 7152845.67 | Avg Fit

...

Generation 99/100 | Best Fitness: 5002260.57 | Avg Fitness: 5133366.21
Generation 100/100 | Best Fitness: 5002260.57 | Avg Fitness: 5199166.49
--- Genetic Algorithm Finished ---

Configuración #45/54: {'population_size': 150, 'generations': 100, 'mutation_rate': 0.15, 'tournament_k': 7}
  Corrida 1/3 (semilla: 42)
Generation 1/100 | Best Fitness: 7923504.60 | Avg Fitness: 9726073.39
Generation 2/100 | Best Fitness: 7729684.29 | Avg Fitness: 9250186.60
Generation 3/100 | Best Fitness: 7202072.34 | Avg Fitness: 8882800.24
Generation 4/100 | Best Fitness: 7202072.34 | Avg Fitness: 8581521.83
Generation 5/100 | Best Fitness: 7150839.84 | Avg Fitness: 8556809.70
Generation 6/100 | Best Fitness: 7071204.87 | Avg Fitness: 8310943.96
Generation 7/100 | Best Fitness: 6678097.38 | Avg Fitness: 8158714.78
Generation 8/100 | Best Fitness: 6649276.77 | Avg Fitness: 7818482.53
Generation 9/100 | Best Fitness: 6323539.50 | Avg Fitness: 7593843.40
Generation 10/100 | Best Fitness: 6298939.62 | Avg Fit

Progreso de calibración:  83%|████████▎ | 45/54 [21:44<05:27, 36.39s/it]

Generation 100/100 | Best Fitness: 5259321.45 | Avg Fitness: 5436340.37
--- Genetic Algorithm Finished ---

Configuración #46/54: {'population_size': 150, 'generations': 200, 'mutation_rate': 0.05, 'tournament_k': 3}
  Corrida 1/3 (semilla: 42)
Generation 1/200 | Best Fitness: 7923504.60 | Avg Fitness: 9726073.39
Generation 2/200 | Best Fitness: 7533818.82 | Avg Fitness: 9423470.68
Generation 3/200 | Best Fitness: 7311016.44 | Avg Fitness: 9077321.45
Generation 4/200 | Best Fitness: 7187907.33 | Avg Fitness: 8861452.40
Generation 5/200 | Best Fitness: 6990427.26 | Avg Fitness: 8568069.07
Generation 6/200 | Best Fitness: 6698087.37 | Avg Fitness: 8154620.16
Generation 7/200 | Best Fitness: 6698087.37 | Avg Fitness: 8092137.65
Generation 8/200 | Best Fitness: 6289020.18 | Avg Fitness: 7989129.27
Generation 9/200 | Best Fitness: 6289020.18 | Avg Fitness: 8010687.85
Generation 10/200 | Best Fitness: 6257839.77 | Avg Fitness: 7910414.69
Generation 11/200 | Best Fitness: 6257839.77 | Avg Fit

Progreso de calibración:  85%|████████▌ | 46/54 [22:49<05:57, 44.74s/it]

Generation 200/200 | Best Fitness: 5034701.61 | Avg Fitness: 5082248.59
--- Genetic Algorithm Finished ---

Configuración #47/54: {'population_size': 150, 'generations': 200, 'mutation_rate': 0.05, 'tournament_k': 5}
  Corrida 1/3 (semilla: 42)
Generation 1/200 | Best Fitness: 7923504.60 | Avg Fitness: 9726073.39
Generation 2/200 | Best Fitness: 7642996.83 | Avg Fitness: 9358582.25
Generation 3/200 | Best Fitness: 7281771.48 | Avg Fitness: 9085319.22
Generation 4/200 | Best Fitness: 7160811.03 | Avg Fitness: 8625082.83
Generation 5/200 | Best Fitness: 7122511.89 | Avg Fitness: 8487492.09
Generation 6/200 | Best Fitness: 6542773.20 | Avg Fitness: 8286168.92
Generation 7/200 | Best Fitness: 6236065.44 | Avg Fitness: 8122933.82
Generation 8/200 | Best Fitness: 6236065.44 | Avg Fitness: 8103226.06
Generation 9/200 | Best Fitness: 6236065.44 | Avg Fitness: 7892076.23
Generation 10/200 | Best Fitness: 6045176.25 | Avg Fitness: 7725031.00
Generation 11/200 | Best Fitness: 6045176.25 | Avg Fit

## 3. Análisis comparativo

### 3.1 Metaheurístico vs. Pyomo


##### Calidad de la solución (Valor de la función objetivo)

| Caso   | Pyomo | Algoritmo Genético |
|--------|-------|--------------------|
| Caso 1 |   X   |         X          |
| Caso 2 |   X   |         X          |
| Caso 3 |   X   |         X          |



##### Tiempo de ejecución (segundos)

| Caso   | Pyomo | Algoritmo Genético |
|--------|-------|--------------------|
| Caso 1 |   X   |         X          |
| Caso 2 |   X   |         X          |
| Caso 3 |   X   |         X          |



##### Uso de memoria (MB)

| Caso   | Pyomo | Algoritmo Genético |
|--------|-------|--------------------|
| Caso 1 |   X   |         X          |
| Caso 2 |   X   |         X          |
| Caso 3 |   X   |         X          |



##### Comportamiento al escalar


| Comparación     | Pyomo (∆ Obj.) | Pyomo (∆ Tiempo) | Genético (∆ Obj.) | Genético (∆ Tiempo) |
|-----------------|----------------|------------------|-------------------|---------------------|
| Caso 2 vs Caso 1|       X        |        X         |         X         |         X           |
| Caso 3 vs Caso 1|       X        |        X         |         X         |         X           |




### 3.2 Diferencias cualitativas en las rutas

##### Longitud de rutas

| Caso   | Pyomo | Algoritmo Genético |
|--------|-------|--------------------|
| Caso 1 |   X   |         X          |
| Caso 2 |   X   |         X          |
| Caso 3 |   X   |         X          |


##### Número de vehículos utilizados

| Caso   | Pyomo | Algoritmo Genético |
|--------|-------|--------------------|
| Caso 1 |   X   |         X          |
| Caso 2 |   X   |         X          |
| Caso 3 |   X   |         X          |


##### Balance de carga (NO SE A QUE SE REFIERE PERO PUEDE SER: desviación estándar o rango de carga por vehículo)

| Caso   | Pyomo | Algoritmo Genético |
|--------|-------|--------------------|
| Caso 1 |   X   |         X          |
| Caso 2 |   X   |         X          |
| Caso 3 |   X   |         X          |


### 3.3 Ventajas y desventajas de ambos enfoques

Aunque los algoritmos genéticos (GA) ofrecen claramente la ventaja de rapidez, evidenciamos que el GA completa la búsqueda mucho más rápido que Pyomo bajo cualquier configuración de timeout, su naturaleza heurística implica que no siempre garantizan la factibilidad ni la calidad de la solución, sin un ajuste cuidadoso de hiperparámetros, como tamaño de población, tasas de mutación y cruces, y esquemas de penalización por violaciones de restricciones, el GA puede producir soluciones inviables o subóptimas.

En contraste, Pyomo, al apoyarse en solvers matemáticos exactos, asegura factibilidad siempre que exista una solución, y con suficiente tiempo puede converger al óptimo global, además su salida es determinista y reproducible frente a la aleatoriedad inherente del GA, no obstante esa solidez pagada en tiempo de cómputo puede tornarse prohibitiva en problemas de gran escala o con plazos estrictos, donde el GA resulta más práctico, aunque sacrifica garantías de optimalidad, Pyomo requiere modelar explícitamente todas las restricciones y parámetros, lo que puede aumentar la complejidad de implementación, mientras que el GA admite un diseño más flexible de la función objetivo y restricciones implícitas mediante penalizaciones, por último la integración de criterios de robustez y sensibilidad ante cambios en los datos suele ser más directa en el contexto de programación matemática, aunque nuevos esquemas híbridos que combinan GA para exploración rápida con refinamientos locales de Pyomo pueden capturar lo mejor de ambos mundos.

## 4. Visualización de resultados

### 4.1 Evolución de la mejor solución 

In [ ]:
# aca se grafica la curva de convergencia para los 3 casos

### 4.2 Rutas finales

In [ ]:
# GRAFICO DE rutas finales superpuestas en el plano para metaheurístico y Pyomo.

### 4.3 Otros resultados

In [ ]:
# ACA SE GRAFICA histogramas o cajas de distribución de cargas y longitudes de ruta